In [36]:
import pandas as pd

from pulp import LpProblem, LpMaximize, lpSum, LpVariable, LpStatus, LpInteger, LpBinary

initial_data = pd.read_excel("data.xlsx").dropna()

PLAYED_AT_LEAST = 30
data = initial_data[initial_data['PLD'] >= PLAYED_AT_LEAST]

In [37]:
data

,Code,Name,Club,Value,PLD,GLS,ASS,CS,GA,Pts,Position
1,102,B. Leno,ARS,3.5,35,0,0,11,37,20,GK
2,103,E. Martinez,AV,3.2,38,0,0,15,46,22,GK
9,110,N. Pope,BUR,3.7,32,0,0,11,37,17,GK
11,112,E. Mendy,CHE,3.9,31,0,0,16,25,38,GK
15,116,V. Guaita,CP,3.0,37,0,0,8,64,-11,GK
...,...,...,...,...,...,...,...,...,...,...,...
427,641,S. Mane,LIV,8.1,31,11,11,0,0,55,ST
437,651,M. Rashford,MU,7.8,34,11,11,0,0,55,ST
444,658,C. Adams,SOT,6.3,31,9,5,0,0,37,ST
449,663,H. Kane,TOT,9.1,35,23,14,0,0,97,ST


In [38]:
# Create the data groups

PLAYERS = data['Name'].tolist()

TEAMS = {
    team: data[data['Club'] == team]['Name'].tolist() for team in data['Club'].unique().tolist()
}

POSITIONS = {
    position: data[data['Position'] == position]['Name'].tolist() for position in data['Position'].unique().tolist()
}

values = {
    player: data[data['Name'] == player].iloc[0]['Value'] for player in PLAYERS
}

points = {
    player: data[data['Name'] == player].iloc[0]['Pts'] for player in PLAYERS
}


In [39]:
# Disallowed teams for defensive players
DISALLOWED_DF_TEAMS = ['BHA', 'BRE', 'SOT', 'WAT', 'WOL', 'CP', 'NOR', 'LEE', 'NEW']
DF_POSITIONS = ['GK', 'DF']

# Disallowed players
DISALLOWED_PLAYERS = ['J. Lingard', 'H. Kane']

df_players = POSITIONS['DF']
gk_players = POSITIONS['GK']
COMBINED_DF_PLAYERS = df_players + gk_players



In [42]:
# Create problem

model = LpProblem("Fantasy", LpMaximize)

# variables
position_constraints = {
    'GK': 1,
    'DF': 4,
    'MF': 4,
    'ST': 2
}

# Create player variables

player_vars = LpVariable.dicts("PlayerVariables", [(i) for i in PLAYERS], 0, 1, LpBinary)

## CONSTRAINTS

# Value constraint
model += lpSum(player_vars[(i)] * values[i] for i in PLAYERS) <= 50.0

# Team constraint
for team, players in TEAMS.items():
    model += lpSum(player_vars[(i)] for i in players) <= 2

# Disallowed defensive combinations
for team in DISALLOWED_DF_TEAMS:
    if team in TEAMS.keys():
        model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) == 0

# Only one defensive player per team to reduce correlation
for team in TEAMS:
    if team in TEAMS.keys():
        model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) <= 1

# Disallowed players
for player in DISALLOWED_PLAYERS:
    if player in PLAYERS:
        model += player_vars[(player)] == 0

    
# Position constraint
for position, players in POSITIONS.items():
    model += lpSum(player_vars[(i)] for i in players) == position_constraints[position]
    
## OBJECTIVE FUNCTION
model += lpSum(player_vars[(i)] * points[i] for i in PLAYERS)


## SOLVE
model.solve()
    

1

In [43]:
LpStatus[model.status]

'Optimal'

In [44]:
total_sum = 0
for i in PLAYERS:
    if player_vars[(i)].varValue == 1:
        print(f'Player: {i}, Points: {points[i]}')
        total_sum += points[i]
total_sum

Player: E. Mendy, Points: 38
Player: M. Targett, Points: 36
Player: R. Dias, Points: 46
Player: A. Wan-Bissaka, Points: 40
Player: A. Cresswell, Points: 37
Player: E. Eze, Points: 28
Player: J. Harrison, Points: 42
Player: B. Fernandes, Points: 84
Player: T. Soucek, Points: 34
Player: O. Watkins, Points: 60
Player: P. Bamford, Points: 71


516